## Publication-Ready Output Tables via `Stargazer`

The [Stargazer](https://github.com/StatsReporting/stargazer) Python package support regression output for models from `PyFixest`. For details on stargazer's functionality, please take a look at the [stargazer examples](https://github.com/StatsReporting/stargazer/blob/master/examples.ipynb). 

We have opened a PR to the [Stargazer repo](https://github.com/StatsReporting/stargazer/pull/105). Until it is merged, you can download the dev version from `py-econometrics` by typing 

```bash
pip install git+https://github.com/py-econometrics/stargazer.git
```

In a first step, we fit a few regression models via `PyFixest's` multiple estimation syntax: 

In [1]:
import numpy as np
from stargazer.stargazer import LineLocation, Stargazer

import pyfixest as pf

fit = pf.feols("Y ~ csw(X1, X2) | csw0(f1, f2)", data=pf.get_data())

We can now produce a html table with the results of the three regressions by simply passing a list of `Feols`, `Fepois` or `Feiv` objects to `Stargazer`:

In [2]:
result_table = Stargazer(fit.to_list())
result_table

Note that we could have achieved a similar result by using the pyfixest native `etable()` function. 

In [3]:
pf.etable(fit.to_list())

                           est1               est2               est3               est4               est5               est6
------------  -----------------  -----------------  -----------------  -----------------  -----------------  -----------------
depvar                        Y                  Y                  Y                  Y                  Y                  Y
------------------------------------------------------------------------------------------------------------------------------
Intercept      0.919*** (0.112)   0.889*** (0.108)
X1            -1.000*** (0.085)  -0.993*** (0.082)  -0.949*** (0.069)  -0.950*** (0.067)  -0.919*** (0.065)  -0.924*** (0.061)
X2                               -0.176*** (0.022)                     -0.174*** (0.018)                     -0.174*** (0.015)
------------------------------------------------------------------------------------------------------------------------------
f1                            -                  -          

Additionally, we can ask `Stargazer` to output the results in LaTeX format:

In [4]:
print(result_table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable: Y}} \
\cr \cline{2-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Intercept & 0.919$^{***}$ & 0.889$^{***}$ & & & & \\
& (0.112) & (0.108) & & & & \\
 X1 & -1.000$^{***}$ & -0.993$^{***}$ & -0.949$^{***}$ & -0.950$^{***}$ & -0.919$^{***}$ & -0.924$^{***}$ \\
& (0.085) & (0.082) & (0.069) & (0.067) & (0.065) & (0.061) \\
 X2 & & -0.176$^{***}$ & & -0.174$^{***}$ & & -0.174$^{***}$ \\
& & (0.022) & & (0.018) & & (0.015) \\
\hline \\[-1.8ex]
 Observations & 998 & 998 & 997 & 997 & 997 & 997 \\
 $R^2$ & 0.123 & 0.177 & 0.437 & 0.489 & 0.609 & 0.659 \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{6}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\
\end{tabular}
\end{table}


And we can add custom statistics!

For example, assume that we want to correct for multiple testing via the Romano-Wolf correction. We can do this as follows:

In [5]:
rwolf_res = pf.rwolf(fit.to_list(), param = "X1", seed = 123, B = 9999)
rwolf_pvalues = np.round(rwolf_res.xs("RW Pr(>|t|)"), 3).to_list()

In [6]:
rwolf_pvalues

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [7]:
result_table.add_line('Fixed Effects', [x._fixef for x in fit.to_list()], LineLocation.FOOTER_TOP)
result_table.add_line('X1: Romano-Wolf P-Value', rwolf_pvalues, LineLocation.FOOTER_TOP)

result_table